In [5]:
import os
import json
import pandas as pd
import traceback

In [79]:
from langchain.chat_models import ChatOpenAI

In [28]:
from dotenv import load_dotenv
from pathlib import Path

env_path = Path("/home/my/Desktop/MCQGEN/.env")
load_dotenv(env_path)

True

In [59]:
KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = os.getenv("OPENAI_API_BASE")

In [60]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

In [63]:
from openai import OpenAI

client = OpenAI(
    base_url=BASE_URL,
    api_key=KEY
)


In [64]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [65]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    }    
}

In [66]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [67]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [68]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [69]:
TEMPLATE2="""
You are an expert english gramarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to avaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.\
If the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz.
"""

In [89]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"],
                                        template=TEMPLATE)

In [90]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [91]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                          output_variables=["quiz", "review"], verbose=True)

In [92]:
file_path = r"/home/my/Desktop/MCQGEN/data.txt"

In [109]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [110]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [111]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"
 

In [112]:
with get_openai_callback() as cb:
    response= generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Biology is the scientific study of life and living organisms. It is a broad natural science that encompasses a wide range of fields and unifying principles that explain the structure, function, growth, origin, evolution, and distribution of life. Central to biology are five fundamental themes: the cell as the basic unit of life, genes and heredity as the basis of inheritance, evolution as the driver of biological diversity, energy transformation for sustaining life processes, and the maintenance of internal stability (homeostasis).[1][2]

Biology examines life across multiple levels of organization, from molecules and cells to organisms, populations, and ecosystems. Subdisciplines include molecular biology, physiology, ecology, evolutionary biology, developmental biology, and systematics, among others. Each of these fields applies a range of methods to investigate biological phenomena, including observation, experimentation, and mathematical modeling. Mod

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:Biology is the scientific study of life and living organisms. It is a broad natural science that encompasses a wide range of fields and unifying principles that explain the structure, function, growth, origin, evolution, and distribution of life. Central to biology are five fundamental themes: the cell as the basic unit of life, genes and heredity as the basis of inheritance, evolution as the driver of biological diversity, energy transformation for sustaining life processes, and the maintenance of internal stability (homeostasis).[1][2]

Biology examines life across multiple levels of organization, from molecules and cells to organisms, populations, and ecosystems. Subdisciplines include molecular biology, physiology, ecology, evolutionary biology, developmental biology, and systematics, among others. Each of these fields applies a range of methods to investigate biological phenomena, including observation, experimentation, and mathema

In [113]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")




Total Tokens: 1832
Prompt Tokens: 1232
Completion Tokens: 600
Total Cost: 0.0030480000000000004


In [114]:
response

{'text': 'Biology is the scientific study of life and living organisms. It is a broad natural science that encompasses a wide range of fields and unifying principles that explain the structure, function, growth, origin, evolution, and distribution of life. Central to biology are five fundamental themes: the cell as the basic unit of life, genes and heredity as the basis of inheritance, evolution as the driver of biological diversity, energy transformation for sustaining life processes, and the maintenance of internal stability (homeostasis).[1][2]\n\nBiology examines life across multiple levels of organization, from molecules and cells to organisms, populations, and ecosystems. Subdisciplines include molecular biology, physiology, ecology, evolutionary biology, developmental biology, and systematics, among others. Each of these fields applies a range of methods to investigate biological phenomena, including observation, experimentation, and mathematical modeling. Modern biology is grou

In [115]:
quiz = response.get('quiz')

In [116]:
quiz = json.loads(quiz)

In [117]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [118]:
quiz_table_data

[{'MCQ': 'What is the basic unit of life in biology?',
  'Choices': 'a: Atoms | b: Molecules | c: Cells | d: Organs',
  'Correct': 'c'},
 {'MCQ': 'Which scientist first articulated the theory of evolution by natural selection?',
  'Choices': 'a: Isaac Newton | b: Gregor Mendel | c: Charles Darwin | d: Louis Pasteur',
  'Correct': 'c'},
 {'MCQ': 'What is the basis of inheritance in biology?',
  'Choices': 'a: Genes and heredity | b: Evolution | c: Energy transformation | d: Homeostasis',
  'Correct': 'a'},
 {'MCQ': 'Which field of biology studies the interaction of organisms with their environments?',
  'Choices': 'a: Physiology | b: Ecology | c: Evolutionary biology | d: Developmental biology',
  'Correct': 'b'},
 {'MCQ': 'What is the scientific study of life and living organisms called?',
  'Choices': 'a: Chemistry | b: Geology | c: Biology | d: Physics',
  'Correct': 'c'}]

In [119]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv", index=False)

In [122]:
from datetime import datetime

datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'08_01_2025_17_53_50'